## 导入库和数据

In [5]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
# oftid_train=pd.read_csv(r'D:\Data\TCForNewComer\off_train_sameUser_id_test.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')
samplt=pd.read_csv(r'D:\Data\TCForNewComer\sample_submission.csv')

In [6]:
off_train=off_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
off_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,4,1433,8735,30:5,10,20160214,null
1,4,1469,2902,0.95,10,20160607,null
2,35,3381,1807,300:30,0,20160130,null
3,35,3381,9776,10:5,0,20160129,null
4,35,3381,11951,200:20,0,20160129,null


In [8]:
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test.index=np.arange(0,len(test),1)
test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


In [4]:
on_train=on_train.sort_values(by=['User_id'])
on_train.index=np.arange(0,len(on_train),1)
on_train.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,4,25104,2,100145044,100:10,20160331,null
1,4,45612,1,null,null,null,20160308
2,36,46701,0,null,null,null,20160120
3,64,11200,0,null,null,null,20160526
4,64,29214,0,null,null,null,20160606


In [ ]:
col='Merchant_id'
a=set(on_train[col].values)
b=set(test[col].values)
c=set(oftid_train[col].values)
print(len(b))
print(len(a))
print(len(c))

print(len(a&b))
print(len(b&c))
print(len(c-b))

## 只保留和test的User_id相同的行

In [ ]:
b=set(test['User_id'].values)
c=set(off_train['User_id'].values)
print(len(b))
print(len(c))
print(len(b&c))

#保留off_train中的和test的User_id一样的行
row=[]
i=0
j=10
length=len(c&b)
for inde in (c&b):
    rate=int(i*100/length)
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10
    row.extend(list(off_train[off_train['User_id']==inde].index))
    i+=1
off_train=off_train.loc[row]

off_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\off_train_sameUser_id_test.csv',index=False)

In [ ]:
a=set(on_train['User_id'].values)
b=set(test['User_id'].values)
print(len(b))
print(len(a))
print(len(a&b))

#保留on_train中的和test的User_id一样的行
row=[]
i=0
j=10
length=len(a&b)
for inde in (a&b):
    rate=int(i*100/length)
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10
    row.extend(list(on_train[on_train['User_id']==inde].index))
    i+=1
on_train=on_train.loc[row]

on_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\on_train_sameUser_id_test.csv',index=False)

## 导入处理过以后的数据

In [ ]:
oftid_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\off_train_sameUser_id_test.csv')
ontid_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\on_train_sameUser_id_test.csv')

## 增加一列之前没用优惠券是否购买过相同商品和次数
- 可能是日常用品，或者零食
- 可能是耐用品，然后买了之后就不会再买

In [ ]:
indexs=oftid_train[(oftid_train['Coupon_id']=='null')&(oftid_train['Date']!='null')].index
odinary=oftid_train.iloc[indexs]

In [ ]:
oftid_train.drop(indexs,axis=0,inplace=True)

In [ ]:
oftid_train['ord_buy']=0#增加一列

In [ ]:
odinary.index=np.arange(0,len(odinary),1)
oftid_train.index=np.arange(0,len(oftid_train),1)

In [ ]:
#计算同一件商品半年内普通购买次数
from ipykernel import kernelapp as app
i=0
j=10
length=len(oftid_train)
for index in oftid_train.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=oftid_train.iloc[index]['User_id']
    mid=oftid_train.iloc[index]['Merchant_id']
    if len(odinary[odinary['User_id']==uid].index)!=0:
        for ind in odinary[odinary['User_id']==uid].index:
            if odinary.iloc[ind]['Merchant_id']==mid:
                oftid_train.loc[index,'ord_buy']+=1
    i+=1

In [ ]:
oftid_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oftid_addordinarybuy_train.csv',index=False)

In [ ]:
test['ord_buy']=0#增加一列
test.index=np.arange(0,len(test),1)

#计算同一件商品半年内普通购买次数
i=0
j=10
length=len(test)
for index in test.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=test.iloc[index]['User_id']
    mid=test.iloc[index]['Merchant_id']
    if len(odinary[odinary['User_id']==uid].index)!=0:
        for ind in odinary[odinary['User_id']==uid].index:
            if odinary.iloc[ind]['Merchant_id']==mid:
                test.loc[index,'ord_buy']+=1
    i+=1

In [ ]:
test.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\test_addordinarybuy.csv',index=False)

### 线下领取优惠券张数和不同优惠券的张数和使用数目

In [45]:
import pandas as pd
import numpy as np
or_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oftid_addordinarybuy_train.csv')
or_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\test_addordinarybuy.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
or_train=or_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
or_train.index=np.arange(0,len(or_train),1)
or_test.index=np.arange(0,len(or_test),1)

In [46]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])
    #将满减转化为小数
or_train['Discount_rate']=or_train['Discount_rate'].apply(Discount_rate)

In [47]:
or_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216250 entries, 0 to 216249
Data columns (total 8 columns):
User_id          216250 non-null int64
Merchant_id      216250 non-null int64
Coupon_id        216250 non-null int64
Discount_rate    216250 non-null float64
Distance         216250 non-null object
Date_received    216250 non-null int64
Date             216250 non-null object
ord_buy          216250 non-null int64
dtypes: float64(1), int64(5), object(2)
memory usage: 14.8+ MB


In [49]:
or_train['Coupon_num']=0                   #领取张数
or_train['Coupon_nouse_num']=0             #领取没有使用张数
or_train['Coupon_use_num']=0               #使用张数
or_train['Coupon_use_prob']=0              #使用概率
i=0
j=10
length=len(set(or_train['User_id'].values))
for id in set(or_train['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    mid_Ser=or_train[or_train['User_id']==id]['Date']
    lengths=len(mid_Ser[mid_Ser=='null'])
    or_train.loc[mid_Ser.index,'Coupon_num']=len(mid_Ser)
    or_train.loc[mid_Ser.index,'Coupon_nouse_num']=lengths
    or_train.loc[mid_Ser.index,'Coupon_use_num']=len(mid_Ser)-lengths
    or_train.loc[mid_Ser.index,'Coupon_use_prob']=float(len(mid_Ser)-lengths)/len(mid_Ser)
    i+=1
or_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\or_train.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

In [51]:
or_test['Coupon_num']=0                   #领取张数
or_test['Coupon_nouse_num']=0             #领取没有使用张数
or_test['Coupon_use_num']=0               #使用张数
or_test['Coupon_use_prob']=0              #使用概率
i=0
j=10
length=len(set(or_test['User_id'].values))
for id in set(or_test['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    mid_Ser=or_train[or_train['User_id']==id]['Date']
    if len(mid_Ser)!=0:
        lengths=len(mid_Ser[mid_Ser=='null'])
        test_Ser=or_test[or_test['User_id']==id]
        or_test.loc[test_Ser.index,'Coupon_num']=len(mid_Ser)
        or_test.loc[test_Ser.index,'Coupon_nouse_num']=lengths
        or_test.loc[test_Ser.index,'Coupon_use_num']=len(mid_Ser)-lengths
        or_test.loc[test_Ser.index,'Coupon_use_prob']=float(len(mid_Ser)-lengths)/len(mid_Ser)
    i+=1
or_test.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\or_test.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

### 使用优惠券面额大于100的次数和小于100的次数

In [112]:
import pandas as pd
import numpy as np
oc_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\train.csv')
oc_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\test.csv')
oc_train=oc_train.sort_values(by=['User_id'])
oc_train.index=np.arange(0,len(oc_train),1)
oc_test.index=np.arange(0,len(oc_test),1)

In [353]:
oc_train['Coupon_greaterthan100']=0                   #使用面额大于100的张数
oc_train['Coupon_lessthan100']=0                      #使用面额小于100的张数
oc_train['Coupon_lep100']=0                          #使用面额大于100的张数占使用张数的比重
oc_train['Coupon_grp100']=0                          #使用面额小于100的张数占使用张数的比重
oc_train['Avg_saler_Coupon']=0                       #核销每个商家多少张优惠券
oc_train['Avg_distance']=0                          #用户核销优惠券的平均距离
oc_train['shop_count']=0                            #核销商家数量
oc_train['shop_pro']=0                              #商家优惠券占总的比例
oc_train['Coupon_diffcount']=0                      #不同优惠券的数量
oc_train['Coupon_diffpro']=0                        #不同优惠券的占比
oc_train['same_Coupon_count']=0                      #相同优惠券领取的次数
i=0
j=10
length=len(set(oc_train[oc_train['Date']==1]['User_id'].values))
for id in set(oc_train[oc_train['Date']==1]['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    discount_rate=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]['Discount_rate']
    indexs=oc_train[oc_train['User_id']==id].index
    gr100=len(discount_rate[discount_rate>100])
    le100=len(discount_rate[discount_rate<100])
    use_count=oc_train.loc[discount_rate.index,'Coupon_use_num'].max()
    oc_train.loc[indexs,'Coupon_greaterthan100']=gr100
    oc_train.loc[indexs,'Coupon_lessthan100']=le100
    oc_train.loc[indexs,'Coupon_grp100']=gr100/use_count
    oc_train.loc[indexs,'Coupon_lep100']=le100/use_count
    
    customer=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]
    oc_train.loc[indexs,'shop_count']=len(set(customer['Distance'].values))
    oc_train.loc[indexs,'Coupon_diffcount']=len(set(customer['Discount_rate'].values))
    for dis in set(customer['Distance'].values):                     #不同的商家
        cus=customer[customer['Distance']==dis]
        indexss=oc_train[(oc_train['User_id']==id)&(oc_train['Distance']==dis)].index
        oc_train.loc[indexss,'Avg_saler_Coupon']=len(cus)
        oc_train.loc[indexss,'shop_pro']=len(cus)/len(customer)
        for rat in set(customer['Discount_rate'].values):             #不同优惠额度
            Rate=customer[customer['Discount_rate']==rat]
            indexsss=oc_train[(oc_train['User_id']==id)&(oc_train['Distance']==dis)&(oc_train['Discount_rate']==rat)].index
            oc_train.loc[indexsss,'Coupon_diffpro']=float(len(Rate))/len(cus)
    
    for coupid in set(oc_train[oc_train['User_id']==id]['Coupon_id'].values):
        indexx=oc_train[(oc_train['User_id']==id)&(oc_train['Coupon_id']==coupid)].index
        oc_train.loc[indexx,'same_Coupon_count']=len(indexx)
        
    oc_train.loc[indexs,'Avg_distance']=customer['Distance'].mean()
    i+=1
oc_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_train.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

In [118]:
oc_test['Coupon_greaterthan100']=0                   #使用面额大于100的张数
oc_test['Coupon_lessthan100']=0                      #使用面额小于100的张数
oc_test['Coupon_lep100']=0                          #使用面额大于100的张数占使用张数的比重
oc_test['Coupon_grp100']=0                          #使用面额小于100的张数占使用张数的比重
oc_test['Avg_saler_Coupon']=0                       #核销每个商家多少张优惠券
oc_test['Avg_distance']=0                          #用户核销优惠券的平均距离
oc_test['shop_count']=0                            #核销商家数量
oc_test['shop_pro']=0                              #商家优惠券占总的比例
oc_test['Coupon_diffcount']=0                      #不同优惠券的数量
oc_test['Coupon_diffpro']=0                        #不同优惠券的占比
oc_test['same_Coupon_count']=0                      #相同优惠券领取的次数
i=0
j=10
length=len(set(oc_test['User_id'].values))
for id in set(oc_test['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    discount_rate=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]['Discount_rate']
    indexs=oc_test[oc_test['User_id']==id].index
    gr100=len(discount_rate[discount_rate>100])
    le100=len(discount_rate[discount_rate<100])
    use_count=oc_train.loc[discount_rate.index,'Coupon_use_num'].max()
    oc_test.loc[indexs,'Coupon_greaterthan100']=gr100
    oc_test.loc[indexs,'Coupon_lessthan100']=le100
    if len(discount_rate)!=0:
        oc_test.loc[indexs,'Coupon_grp100']=gr100/use_count
        oc_test.loc[indexs,'Coupon_lep100']=le100/use_count
    
    customer=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]
    oc_test.loc[indexs,'shop_count']=len(set(customer['Distance'].values))
    oc_test.loc[indexs,'Coupon_diffcount']=len(set(customer['Discount_rate'].values))
    for dis in set(oc_test[(oc_test['User_id']==id)]['Distance'].values):                     #不同的商家
        cus=customer[customer['Distance']==dis]
        indexss=oc_test[(oc_test['User_id']==id)&(oc_test['Distance']==dis)].index
        oc_test.loc[indexss,'Avg_saler_Coupon']=len(cus)
        if len(customer)!=0:
            oc_test.loc[indexss,'shop_pro']=float(len(cus))/len(customer)
        
        for rat in set(oc_test[(oc_test['User_id']==id)]['Discount_rate'].values):             #不同优惠额度
            Rate=customer[customer['Discount_rate']==rat]
            indexsss=oc_test[(oc_test['User_id']==id)&(oc_test['Distance']==dis)&(oc_test['Discount_rate']==rat)].index
            if len(cus)!=0:
                oc_test.loc[indexsss,'Coupon_diffpro']=float(len(Rate))/len(cus)
    
    for coupid in set(oc_test[oc_test['User_id']==id]['Coupon_id'].values):
        indexx=oc_test[(oc_test['User_id']==id)&(oc_test['Coupon_id']==coupid)].index
        oc_test.loc[indexx,'same_Coupon_count']=len(indexx)
    if len(customer['Distance'])!=0:
        oc_test.loc[indexs,'Avg_distance']=customer['Distance'].mean()
    i+=1
oc_test.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_test.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

In [12]:
hh['Date_received']=hh['Date_received'].apply(lambda x: int(str(x)[1:]))

In [22]:
for index in kk.index:
    kk.loc[index,'Date_received']=int(str(kk.loc[index,'Date_received'])[1:])

##  商家提供的优惠券种类和张数

### 算有几家店铺，店铺中的商品种类，店铺的优惠券种类和张数，顾客经常使用优惠券的商店，对店铺进行编码，优惠券领取的张数，优惠券使用的张数和占比，发放优惠券的平均面额

In [1]:
import pandas as pd
import numpy as np
os_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_train.csv')
os_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_test.csv')

In [3]:
indx=os_train[os_train['Discount_rate']<1].index
discount=os_train.loc[indx]
os_train=os_train.drop(indx,axis=0)
discount.index=np.arange(0,len(discount),1)
os_train.index=np.arange(0,len(os_train),1)

In [192]:
def shopn(id):                        #计算店铺数量
    for mid in  id:
        indexs=[]
        indexxx.extend(list(os_train[os_train['Merchant_id']==mid].index))
        indexs.extend(list(os_train[os_train['Merchant_id']==mid].index))
#         avg=os_train[os_train['Merchant_id']==mid]['Discount_rate'].mean()

        for uid in set(os_train[os_train['Merchant_id']==mid]['User_id'].values):
            dis=os_train[(os_train['Merchant_id']==mid)&(os_train['User_id']==uid)]['Distance'].max()
            time_min=os_train[(os_train['Merchant_id']==mid)&(os_train['User_id']==uid)]['Date_received'].min()
            time_max=os_train[(os_train['Merchant_id']==mid)&(os_train['User_id']==uid)]['Date_received'].max()
            merchantid=set()
            discount_rate=os_train[(os_train['Merchant_id']==mid)&(os_train['User_id']==uid)]['Discount_rate'].max()
            for ind in os_train[(os_train['User_id']==uid)&(os_train['Distance']==int(dis))].index:
                if (os_train.loc[ind,'Discount_rate']==discount_rate) or ((abs(os_train.loc[ind,'Date_received']-time_min)<7) or (abs(os_train.loc[ind,'Date_received']-time_max)<7)):
                    if ind not in indexxx:
                        merchantid.add(os_train.loc[ind,'Merchant_id'])
                    indexxx.extend([ind])
                    indexs.extend([ind])
            if len(merchantid)!=0:
                indexs.extend(shopn(merchantid))
                indexxx.extend([ind])
    return indexs

In [ ]:
a=set(os_train['Merchant_id'].values).copy()
shop_num=[]
i=0
os_train['shop_id']=0
import winsound
while True:
    global indexxx
    indexxx=[]
    index1=shopn([list(a)[0]])
    shop_num.append(index1)
    os_train.loc[list(set(index1)),'shop_id']=i
    for ind in set(os_train.loc[list(set(index1)),'Merchant_id'].values):
        print(ind)
        a.remove(ind)
    if len(a)==0:
        break
    print('lenp(a):',len(a)/3697,'i:',i)
    winsound.Beep(600,1000)
    i+=1
os_train_shop.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\os_train_shop.csv',index=False)

In [1]:
import pandas as pd
import numpy as np
import time
on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')

In [4]:
on_train#.head(60)

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618
5,14336199,18907,0,null,null,null,20160618
6,14336199,18907,0,null,null,null,20160618
7,14336199,18907,0,null,null,null,20160618
8,14336199,18907,0,null,null,null,20160618
9,14336199,18907,0,null,null,null,20160618


## 增加用券在线购买和普通购买、领取优惠券、点击的列

In [ ]:
test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\test_addordinarybuy.csv')

In [ ]:
oftid_train['online_buy']=0#增加一列
oftid_train['online_col']=0
oftid_train['online_click']=0
test['online_buy']=0#增加一列
test['online_col']=0
test['online_click']=0
# odinary.index=np.arange(0,len(odinary),1)
ontid_train.index=np.arange(0,len(ontid_train),1)
oftid_train.index=np.arange(0,len(oftid_train),1)
test.index=np.arange(0,len(test),1)

In [ ]:
i=0
j=10
length=len(oftid_train)
for index in oftid_train.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=oftid_train.iloc[index]['User_id']
    if len(ontid_train[ontid_train['User_id']==uid].index)!=0:
        for ind in ontid_train[ontid_train['User_id']==uid].index:
            if ontid_train.iloc[ind]['Action']==1:
                oftid_train.loc[index,'online_buy']+=1
            elif ontid_train.iloc[ind]['Action']==2:
                oftid_train.loc[index,'online_col']+=1
            else:
                oftid_train.loc[index,'online_click']+=1
    i+=1

In [ ]:
oftid_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\ofon_coupod_buy_train.csv',index=False)

In [ ]:
i=0
j=10
length=len(test)
for index in test.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=test.iloc[index]['User_id']
    if len(ontid_train[ontid_train['User_id']==uid].index)!=0:
        for ind in ontid_train[ontid_train['User_id']==uid].index:
            if ontid_train.iloc[ind]['Action']==1:
                test.loc[index,'online_buy']+=1
            elif ontid_train.iloc[ind]['Action']==2:
                test.loc[index,'online_col']+=1
            else:
                test.loc[index,'online_click']+=1
    i+=1

In [ ]:
test.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\test_ofon_coupod_buy.csv',index=False)

In [52]:
off_train=or_train.copy()
test=or_test.copy()

***

# 导入增加几列以后的数据集

In [295]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
off_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\or_train.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\or_test.csv')

In [296]:
off_train=off_train.sort_values(by=['User_id'])
off_train['Date_received']=off_train['Date_received'].apply(lambda x: str(x))
off_train.index=np.arange(0,len(off_train),1)
off_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,ord_buy,Coupon_num,Coupon_nouse_num,Coupon_use_num,Coupon_use_prob
0,215,129,8944,25.0,1,20160524,null,1,1,1,0,0.0
1,316,7974,8952,40.0,0,20160430,null,0,1,1,0,0.0
2,417,3381,11951,180.0,0,20160227,null,0,5,4,1,0.2
3,417,450,8555,25.0,1,20160203,null,0,5,4,1,0.2
4,417,775,5435,25.0,0,20160329,20160412,1,5,4,1,0.2


In [297]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])
#将满减转化为小数
test['Discount_rate']=test['Discount_rate'].apply(Discount_rate)
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test.index=np.arange(0,len(test),1)
test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,ord_buy,Coupon_num,Coupon_nouse_num,Coupon_use_num,Coupon_use_prob
0,4129537,450,9983,25.0,1,20160712,0,2,2,0,0.0
1,6949378,1300,3429,25.0,null,20160706,0,1,0,1,1.0
2,2166529,7113,6928,180.0,5,20160727,0,1,1,0,0.0
3,2166529,7113,1808,90.0,5,20160727,0,1,1,0,0.0
4,6172162,7605,6500,29.0,2,20160708,0,1,1,0,0.0


## 选取间隔日期大于15天，标记为负样本

In [298]:
p_train=off_train[off_train['Date']!='null'].copy()#正样本
n_train=off_train[off_train['Date']=='null'].copy()#负样本
p_train.index=np.arange(0,len(p_train),1)
n_train.index=np.arange(0,len(n_train),1)
#转换为时间格式
p_train['Date_received_copy']=p_train['Date_received'].copy()

p_train['Date_received_copy']=pd.to_datetime(p_train['Date_received_copy'])
p_train['Date']=pd.to_datetime(p_train['Date'])

p_train['date']=(p_train['Date']-p_train['Date_received_copy']).astype('timedelta64[D]')
p_train.loc[p_train[p_train['date']<=15].index,'date']=1
p_train.loc[p_train[p_train['date']>15].index,'date']=0
p_train['Date']=p_train['date'].copy()
p_train.drop(['date','Date_received_copy'],axis=1,inplace=True)
p_train['Date']=p_train['Date'].astype(np.int64)

In [299]:
#转换为星期几x.weekday()+1
test['Date_received']=(test['Date_received'].apply(lambda x: int(x[5:8]))).copy()
#正样本为1，负样本为0
n_train['Date']=0
train=pd.concat([p_train,n_train],axis=0)
train['Date_received']=(train['Date_received'].apply(lambda x: int(x[5:8]))).copy()
train['Date_received']=train['Date_received'].astype(np.int64)
train=train.sort_values(by=['User_id'])
train.index=np.arange(0,len(train),1)

## 转化满减为小数折扣

In [300]:
train=train.replace('null',np.nan)
test=test.replace('null',np.nan)

In [301]:
train=train.fillna(method='ffill')

In [302]:
train['Distance']=train['Distance'].astype(np.int64)
train['Coupon_id']=train['Coupon_id'].astype(np.int64)

In [303]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216250 entries, 0 to 216249
Data columns (total 12 columns):
User_id             216250 non-null int64
Merchant_id         216250 non-null int64
Coupon_id           216250 non-null int64
Discount_rate       216250 non-null float64
Distance            216250 non-null int64
Date_received       216250 non-null int64
Date                216250 non-null int64
ord_buy             216250 non-null int64
Coupon_num          216250 non-null int64
Coupon_nouse_num    216250 non-null int64
Coupon_use_num      216250 non-null int64
Coupon_use_prob     216250 non-null float64
dtypes: float64(2), int64(10)
memory usage: 21.4 MB


## 不同类型数据的影响

In [ ]:
train['Merchant_id']=train['Merchant_id'].astype(np.object)
train['User_id']=train['User_id'].astype(np.object)
train['Date_received']=train['Date_received'].astype('category')
test['Merchant_id']=test['Merchant_id'].astype(np.object)
test['Coupon_id']=test['Coupon_id'].astype(np.object)
test['Distance']=test['Distance'].astype(np.object)
test['Date_received']=test['Date_received'].astype('category')

In [ ]:
test['User_id']=test['User_id'].astype(np.object)

### 填充测试集的空缺值

In [304]:
test=test.fillna(method='ffill')

In [305]:
test['Merchant_id']=test['Merchant_id'].astype(np.int64)
test['Coupon_id']=test['Coupon_id'].astype(np.int64)
test['Distance']=test['Distance'].astype(np.int64)

In [306]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113640 entries, 0 to 113639
Data columns (total 11 columns):
User_id             113640 non-null int64
Merchant_id         113640 non-null int64
Coupon_id           113640 non-null int64
Discount_rate       113640 non-null float64
Distance            113640 non-null int64
Date_received       113640 non-null int64
ord_buy             113640 non-null int64
Coupon_num          113640 non-null int64
Coupon_nouse_num    113640 non-null int64
Coupon_use_num      113640 non-null int64
Coupon_use_prob     113640 non-null float64
dtypes: float64(2), int64(9)
memory usage: 10.4 MB


In [ ]:
#行最大最小归一化
XX=train.copy()
for col in ['Merchant_id','Coupon_id']:
    max=XX[col].max()
    min=XX[col].min()
    XX[col]=XX[col].apply(lambda x: ((x-min)/(max-min)))

In [ ]:
XX.head()

In [ ]:
cor=XX.corr()
cor[cor<0.01]=0
cor

## 距离统计分布

In [ ]:
import seaborn as sns 
%matplotlib inline

sns.countplot(x='Distance',hue='Date',data=train)

In [ ]:
import seaborn as sns
%matplotlib inline

sns.countplot(x='Date_received',hue='Date',data=train)

In [ ]:
import seaborn as sns 
%matplotlib inline
plt.figure(figsize=(10,6))
sns.countplot(x='Discount_rate',hue='Date',data=train)
plt.xticks(rotation=90)
plt.show()

In [ ]:
train.head()

In [ ]:
a=set(train['Coupon_id'].values)
c=set(test['Coupon_id'].values)
len(c)

In [357]:
corr=oc_train.corr()
corr

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,ord_buy,Coupon_num,Coupon_nouse_num,...,Coupon_lessthan100,Coupon_lep100,Coupon_grp100,Avg_saler_Coupon,Avg_distance,shop_count,shop_pro,Coupon_diffcount,Coupon_diffpro,same_Coupon_count
User_id,1.000000,0.004005,-0.001469,-0.003272,0.000964,-0.002444,0.000421,-0.006952,-0.017397,-0.011613,...,-0.016907,-0.003299,-0.000303,-0.016830,0.006692,-0.002044,-0.003171,-0.003258,-0.002287,-0.017042
Merchant_id,0.004005,1.000000,0.138292,-0.008626,-0.003292,-0.137270,0.057725,0.080102,0.078623,0.044146,...,0.076124,0.063998,0.007644,0.080026,0.031900,0.060801,0.087671,0.074345,0.074265,0.073047
Coupon_id,-0.001469,0.138292,1.000000,0.134820,0.045436,-0.145048,0.026200,0.055565,0.054967,0.039055,...,0.042967,0.034292,0.008179,0.042435,0.021788,0.034097,0.041426,0.041838,0.022016,0.055806
Discount_rate,-0.003272,-0.008626,0.134820,1.000000,0.123656,-0.287202,-0.151889,-0.143597,-0.042159,0.008198,...,-0.081950,-0.150810,0.069996,-0.082522,-0.046208,-0.130015,-0.170790,-0.120311,-0.189446,-0.096258
Distance,0.000964,-0.003292,0.045436,0.123656,1.000000,-0.168149,-0.126451,-0.184345,-0.044554,0.005429,...,-0.075609,-0.124763,-0.007844,-0.092196,0.195608,-0.115968,-0.212422,-0.125490,-0.147698,-0.081809
Date_received,-0.002444,-0.137270,-0.145048,-0.287202,-0.168149,1.000000,0.092681,0.090143,-0.076489,-0.147139,...,0.055123,0.084282,-0.023080,0.061119,0.048020,0.066962,0.120387,0.063099,0.123834,0.056793
Date,0.000421,0.057725,0.026200,-0.151889,-0.126451,0.092681,1.000000,0.269598,0.160456,-0.061029,...,0.338986,0.519031,0.062136,0.347783,0.187342,0.504734,0.624277,0.497702,0.726871,0.307410
ord_buy,-0.006952,0.080102,0.055565,-0.143597,-0.184345,0.090143,0.269598,1.000000,0.440441,0.270713,...,0.372477,0.260801,0.035590,0.404596,0.008838,0.259122,0.346577,0.347479,0.303077,0.479700
Coupon_num,-0.017397,0.078623,0.054967,-0.042159,-0.044554,-0.076489,0.160456,0.440441,1.000000,0.801607,...,0.649425,0.299203,0.066784,0.641447,0.086869,0.333093,0.279390,0.472562,0.199977,0.619247
Coupon_nouse_num,-0.011613,0.044146,0.039055,0.008198,0.005429,-0.147139,-0.061029,0.270713,0.801607,1.000000,...,0.076443,0.116678,0.056039,0.081559,0.036772,0.144314,0.081627,0.208460,0.004079,0.229332


In [ ]:
combine=[train,test]
for data in combine:
    data.drop(['Date_received'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
a=set(train['Merchant_id'].values)
c=set(test['Merchant_id'].values)
len(c)

In [ ]:
len(a&c)

# 训练模型

In [79]:
K=10
from sklearn.metrics import roc_auc_score,auc
from time import sleep
import winsound
from sklearn.model_selection import StratifiedKFold
# kf = KFold(n_splits = K,random_state = 1,shuffle = True)
kf=StratifiedKFold(n_splits = K,random_state = 90,shuffle = True)
X=hhh.drop(['Date'],axis=1)
y=hhh['Date']

In [53]:
test=TEST.drop(['Date'],axis=1)
test_y=TEST['Date']

## 归一化

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
x_predict=test
# 1. min max scaler
min_max_scaler = MinMaxScaler()
x_predict_min_max_scaled = min_max_scaler.fit_transform(x_predict)

# 2. standard scaler
std_scaler = StandardScaler()
x_predict_std_scaled = std_scaler.fit_transform(x_predict)

In [ ]:
# 1. min max scaler
min_max_scaler = MinMaxScaler()
# x_train_min_max_scaled = min_max_scaler.fit_transform(x_train)
# x_test_min_max_scaled = min_max_scaler.transform(x_test)
X_test_min_max_scaled=min_max_scaler.fit_transform(X)#整个数据集
# 2. standard scaler
std_scaler = StandardScaler()
# x_train_std_scaled = std_scaler.fit_transform(x_train)
# x_test_std_scaled = std_scaler.transform(x_test)
X_test_std_scaled=std_scaler.fit_transform(X)

In [123]:
import xgboost as xgb
#全部  0.5552 learning_rate=0.09
model=xgb.XGBClassifier(n_estimators=200,
                        max_depth=2,#6
                        objective="binary:logistic",
                        learning_rate=0.09, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.4,
                        scale_pos_weight=1.6,
                        gamma=9,
                        seed=100,
                        reg_alpha=8,
                        reg_lambda=1.3)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
print('accuracy:',model.score(X,y))
pred_test=model.predict_proba(kkk)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

accuracy: 0.914649710983
Avg_auc for full training set: 0.849323214039


In [122]:
def Avg_auc(pred):
    aucc=0
    i=0
    df=hhh.copy()
    df['y_valid_pred']=pred
    for j in list(set(df['Coupon_id'].values)):
        df_1=df[df['Coupon_id']==j]
        if len(np.unique(df_1['Date']))==1:
            continue
        aucc=aucc+roc_auc_score(df_1['Date'],df_1['y_valid_pred'])
        i+=1
    aucc=aucc/i                                                                      #len(set(df['Coupon_id'].values))
    df.drop(['y_valid_pred'],axis=1,inplace=True)
    return aucc

In [81]:
def Avg_testauc(pred):
    aucc=0
    i=0
    df=TEST.copy()
    df['y_valid_pred']=pred
    for j in list(set(df['Coupon_id'].values)):
        df_1=df[df['Coupon_id']==j]
        if len(np.unique(df_1['Date']))==1:
#             print(j)
            continue
        aucc=aucc+roc_auc_score(df_1['Date'],df_1['y_valid_pred'])
        i+=1
    aucc=aucc/i                                                     #len(set(df['Coupon_id'].values))
    df.drop(['y_valid_pred'],axis=1,inplace=True)
    return aucc

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

In [87]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=400)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
pred_y=model.predict(X)
print('accuracy:',model.score(X,y))
rf_pred_test=model.predict_proba(kkk)[:,1]
# print('Avg_auc for full training set:',Avg_auc(pred))

accuracy: 0.999486705202


In [ ]:
from sklearn.ensemble import AdaBoostClassifier#星期0.626
model=AdaBoostClassifier(n_estimators=400,learning_rate=1,random_state=100)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
print('accuracy:',model.score(X,y))
adb_pred_test=model.predict_proba(kkk)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))
# print('Avg_auc for full test set:',Avg_testauc(pred_test))

In [ ]:
y_test_pred=0
y_valid_pred=0*y
fit_model=[]
# XMIN=pd.DataFrame(X_test_std_scaled,columns=['Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received'])
for i,(train_index,test_index) in enumerate(kf.split(X,y)):
    y_train,y_valid=y.iloc[train_index],y.iloc[test_index]
    x_train,x_valid=X.iloc[train_index],X.iloc[test_index]
    print('\nFlod:',i,end=':')
    fitted_model=model.fit(x_train,y_train)
    pred= fitted_model.predict_proba(x_valid)[:,1]
    y_valid_pred.iloc[test_index]=pred
    print(pd.DataFrame(pred).head(1))
    
    y_test_pred+=fitted_model.predict_proba(test)[:,1]
    fit_model.append(fitted_model)
    winsound.Beep(600,1000)
    sleep(1)
y_test_pred/=K
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [85]:
xgb=pred_test

In [89]:
rf=rf_pred_test

In [90]:
adb=adb_pred_test

In [ ]:
h=pd.DataFrame()
h['xgb']=xgb
h['rf']=rf
h['adb']=adb
h['avg']=h.mean(axis=1)

In [94]:
tt=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')

In [95]:
sub=pd.DataFrame()
sub['User_id']=tt['User_id']
sub['Coupon_id']=tt['Coupon_id']
sub['Date_received']=tt['Date_received']

In [96]:
sub['Probability']=xgb

In [97]:
sub.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\xgbpred_result.csv',float_format='%.1f',index=False)

In [ ]:
y_train['a']=np.random.normal(len(y_train))

In [ ]:
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 1, 0, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
y_true.astype(np.int64)

In [ ]:
train.head()